# Using `numba.jit` to speedup the computation of the Euclidean distance matrix 

In this notebook we implement a function to compute the Euclidean distance matrix using Numba's *just-in-time* compilation decorator. We compare it with the NumPy function we wrote before.

We will use two Numba functions here: The decorator ` @numba.jit` and `numba.prange`.

In [1]:
import numpy as np
import numba

In [2]:
def euclidean_numpy(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N,) numpy array
    y: (N,) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """

    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[:, np.newaxis].T

    xy = np.dot(x, y.T)

    return np.abs(x2 + y2 - 2. * xy)

In [3]:
@numba.jit(nopython=True)
def euclidean_numba1(x, y):
    """Euclidean square distance matrix using pure loops
    and no NumPy operations
    """
    num_samples, num_feat = x.shape
    dist_matrix = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in numba.prange(num_feat):
                r += (x[i][k] - y[j][k])**2
            dist_matrix[i][j] = r
    return dist_matrix


@numba.jit(nopython=True)
def euclidean_numba2(x, y):
    """Euclidean square distance matrix using loops
    and the `numpy.dot` operation
    """
    num_samples, num_feat = x.shape
    dist_matrix = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in numba.prange(num_samples):
            dist_matrix[i][j] = ((x[i] - y[j])**2).sum()
    return dist_matrix

In [4]:
# Let's check that they all give the same result
a = 10. * np.random.random([100, 10])

print(np.abs(euclidean_numpy(a, a) - euclidean_numba1(a, a)).max())
print(np.abs(euclidean_numpy(a, a) - euclidean_numba2(a, a)).max())

3.268496584496461e-13
3.268496584496461e-13


Our Numba implementations can be faster than the NumPy one for a list of small vectors. However, with larger vectors, the NumPy implementation is faster:

In [5]:
nsamples = 100
nfeat = 3

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)
%timeit euclidean_numba2(x, x)

77.8 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
29.2 µs ± 21.6 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.35 ms ± 5.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
nsamples = 100
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)
%timeit euclidean_numba2(x, x)

83.2 µs ± 212 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
390 µs ± 261 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.98 ms ± 1.83 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In a more realistic case, our NumPy implementation is much faster:

In [7]:
nsamples = 5000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)

490 ms ± 310 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.11 s ± 676 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


As homework, try the `numba.jit` decorator with the following implementations. Does it always work?


```python
def euclidean_loop0(x, y):
    """Euclidean square distance matrix
    
    Inputs:
    x: (N,) numpy array
    y: (N,) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """
    
    num_samples, num_feat = x.shape
    dist_matrix = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in range(num_feat):
                r += (x[i][k] - y[j][k])**2
            dist_matrix[i][j] = r
    return dist_matrix


def euclidean_loop1(x, y):
    """Euclidean square distance matrix
    
    Inputs:
    x: (N,) numpy array
    y: (N,) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """
    
    num_samples, num_feat = x.shape
    dist_matrix = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            dist_matrix[i][j] = ((x[i] - y[j])**2).sum()
    return dist_matrix


def euclidean_loop2(x, y):
    """Euclidean square distance matrix
    
    Inputs:
    x: (N,) numpy array
    y: (N,) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """
    num_samples = x.shape[0]
    dist_matrix = np.zeros((num_samples, num_samples))
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            diff = xi - yj
            dist_matrix[i][j] = np.dot(diff, diff)
    return dist_matrix


def euclidean_loop3(x, y):
    """Euclidean square distance matrix
    
    Inputs:
    x: (N,) numpy array
    y: (N,) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """
    num_samples = x.shape[0]
    dist_matrix = [[np.dot(xi - yj, xi - yj) for xi in x] for yj in y]

    return np.array(dist_matrix)
```